In [71]:
#Caricare le librerie
import tweepy
import pandas as pd
from tqdm import tqdm
import re

In [72]:
consumer_key= "R8zEjULDipNcbMMeGNvnZcbp6"
consumer_secret= "6V856b39nGvS5bgKj9Mf15hc6FEiFnhSlaC8A7G81ZIS7bPxha"
access_token= "1054459819145510914-zRzbWMLQui10gwAL0Rh4IRAbPsZXLT"
access_token_secret= "kZj5myoG661SRnobImDChIIer1b6jHdaDmjqvygKds03T"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [73]:
#Scaricare in un dataframe i tweet in base ad un hashtag (until="2021-12-21")
#NB: hasattr(obj, name, /) Return whether the object has an attribute with the given name.

hashtags=["#NoGreenPass","#vaccinatevi","#casapound"]
list_tweet=[]
list_originals=[]
list_rt_id=[]
list_retweetted=[]


for hashtag in hashtags:
  next_cursor=None
  n=5000
  pbar = tqdm(total=n)
  while n>0:
    for tweet in tweepy.Cursor(api.search_tweets,q=hashtag,lang="it", tweet_mode='extended').items(n): #,cursor=next_cursor

      #gestione dei retweet con try-except
      try:
        list_originals.append(tweet.retweeted_status.id) #creo lista dei retweets id 
        list_rt_id.append(tweet.retweeted_status.user.id)

        #lista tweets che sono stati retweettati
        try: 
          rt_text=tweet.retweeted_status.full_text
        except AttributeError:
          rt_text=tweet.retweeted_status.text

        list_retweetted.append([tweet.retweeted_status.created_at, tweet.retweeted_status.id,tweet.retweeted_status.user.id, rt_text, 
                         tweet.retweeted_status.favorite_count, tweet.retweeted_status.retweet_count, 
                         tweet.retweeted_status.user.screen_name]) 
      except:
        list_originals.append(0)
        list_rt_id.append(0) 

      try: 
        tweet_text=tweet.full_text
      except AttributeError:
        tweet_text=tweet.text

      list_tweet.append([tweet.created_at, tweet.id,tweet.user.id, tweet_text, 
                         tweet.favorite_count, tweet.retweet_count, 
                         tweet.user.location,tweet.user.screen_name,
                         hasattr(tweet, "retweeted_status"),
                         tweet.in_reply_to_screen_name])
      n -=1
      #next_cursor=tweet
      pbar.update(n=1)









  0%|          | 0/5000 [55:17<?, ?it/s]












































  7%|▋         | 345/5000 [11:39:00<157:11:35, 121.57s/it]






















































































































































































































































































































Rate limit reached. Sleeping for: 781






















































































































































































































































































































  4%|▎         | 181/5000 [00:08<03:39, 21.91it/s]

100%|██████████| 5000/5000 [29:45<00:00,  2.80it/s]  
Rate limit reached. Sleeping for: 783
Rate limit reached. Sleeping for: 787


In [75]:
#creo df dei tweet
tweets = pd.DataFrame(list_tweet, 
                      columns=['date','id','user_id','text','like','n_rt','location','author',
                               "retweet","reply_name"])


tweets["id_retweet"]=list_originals
tweets["user_id_retweet"]=list_rt_id


#creo df dei retweet
retweetted = pd.DataFrame(list_retweetted, 
                      columns=['date','id','author_id','text','like','n_rt','author'])    

retweetted['colours']='#aaaacc'

tweets.head()

,date,id,user_id,text,like,n_rt,location,author,retweet,reply_name,id_retweet,user_id_retweet
0,2021-12-29 09:12:07+00:00,1476118824604905473,354747398,RT @singerindubai: Ho appena visto #DontLookUp...,0,1,,MarcG_69,True,None,1476028900312551426,2963478049
1,2021-12-29 09:11:31+00:00,1476118672624209926,1355767833922306050,I Germania si moltiplicano gli scontri in vari...,0,0,,PGrezzo,False,None,0,0
2,2021-12-29 09:11:11+00:00,1476118589543522308,885303356,RT @Lorenzo62752880: Anche il sindaco di Veron...,0,106,,rekotc,True,None,1475926285365006345,1364298062919794688
3,2021-12-29 09:10:04+00:00,1476118309527597060,19592755,RT @Lorenzo62752880: Anche il sindaco di Veron...,0,106,,xposeidone,True,None,1475926285365006345,1364298062919794688
4,2021-12-29 09:09:03+00:00,1476118053633105922,494358540,RT @1nessuno100mil2: In Italia si respira una ...,0,1,,the_bullet_xxx,True,None,1476117524446130180,1296390431190679553


In [76]:
tweets_2=tweets.merge(retweetted[['id','text']],left_on='id_retweet',right_on='id',how='left')[['date','id_x','user_id','text_x','author','retweet','id_retweet','user_id_retweet','text_y']].rename(columns={'text_y':'text_complete'})

In [77]:
tweets_2=tweets_2[tweets_2.text_complete.notnull()].drop_duplicates().reset_index(drop=True)

In [78]:
#arricchimento df tweets
tags_column=[]
weigth_column=[]
for text in tweets_2.text_complete:
    tags_list=[]
    for tag in hashtags:
        finder=re.findall(tag,text)
        if finder != []:
            tags_list.append(finder[0])

    if tags_list==["#NoGreenPass"]:
        weigth_column.append('#fb1239')
    elif tags_list==["#vaccinatevi"]:
        weigth_column.append('#007474')
    elif tags_list==["#casapound"]:
        weigth_column.append('#042069')
    else:
        weigth_column.append('#aaaacc')

    tags_column.append(tags_list)
    
tweets_2['hashtags']=tags_column
tweets_2['colours']=weigth_column

In [79]:
retweetted=retweetted.drop_duplicates().reset_index(drop=True)

In [80]:
#selezione dei tweet più retwettati 
retweetted.sort_values("id",ascending=False).drop_duplicates()

,date,id,author_id,text,like,n_rt,author,colours
514,2021-12-29 09:27:57+00:00,1476122809386553345,1046084474218455040,Si chiama vita. #vaccinatevi e non rompete i c...,1,1,UnitedPlanetFe1,#aaaacc
2,2021-12-29 09:06:57+00:00,1476117524446130180,1296390431190679553,"In Italia si respira una tensione enorme, i n...",4,1,1nessuno100mil2,#aaaacc
6,2021-12-29 08:42:32+00:00,1476111379539148801,757521873743798272,"Quindi, i signori al governo vogliono spingerv...",3,2,SStef_III,#aaaacc
12,2021-12-29 08:26:18+00:00,1476107295062904834,1449347891861106688,Qualcuno mi sa spiegare questa affermazione da...,1,1,Ilconformista20,#aaaacc
515,2021-12-29 07:49:33+00:00,1476098044995280896,1269002722264653831,Sbagliato pensare che #Omicron sia innocuo: lo...,39,5,Fabio_DeBunker,#aaaacc
...,...,...,...,...,...,...,...,...
394,2021-07-21 20:06:41+00:00,1417939106194202626,1302616789348691968,Ce ne fosse UNO che parli di cure.Ormai è ac...,94,47,OrtigiaP,#aaaacc
316,2021-07-15 19:43:38+00:00,1415758978387267584,857713205547814912,La libertà sembra ormai destinata a tramontare...,16,6,mimicetta,#aaaacc
535,2021-06-06 18:02:51+00:00,1401600488886394884,1253699877931872256,"Metti un giorno a Roma.\n\nTu chiamale, se vuo...",282,33,__rebel8,#aaaacc
533,2021-05-11 14:52:02+00:00,1392130382213038081,987756923129876481,Tra gli indagati per gli insulti a #Mattarella...,1996,513,PBerizzi,#aaaacc


##### Vado a vedere quali sono i tweet più retwittati e quante volte sono stati ritwittati dalla nostra rete (usando SQL)

In [83]:
#Open the terminal and run-   pip install U pandasql
#Open your Jupyter Notebook and in any cell run-    !pip install -U pandasql
import pandasql as ps
q1 = """SELECT id,like,n_rt, author,count() FROM retweetted group by id order by n_rt desc"""
print(ps.sqldf(q1, locals()))

                      id  like  n_rt           author  count()
0    1472593787633426432  2318  1315  cristian_paroli        1
1    1446891133762785285  2685  1153   Giorgiolaporta        1
2    1457815379045060608  3035  1106  strange_days_82        1
3    1463423354187169797  2211  1000  strange_days_82        1
4    1434976371336433667  1935   797       EmeiMarkus        1
..                   ...   ...   ...              ...      ...
555  1469704725314609152     1     1       Gia_Felino        1
556  1467093215627124736     3     1  pecorefannobooh        1
557  1460279676635516928     5     1       MarcoAlici        1
558  1457362046643748867     0     1       angheran70        1
559  1476051357090074624     3     0        Serena589        1

[560 rows x 5 columns]


#### Creazione input Grephi tramite scrittura file .csv
Da rivedere:
- Abbiamo creato nodi con l'id del tweet, non per utente
- Si possono pesare gli archi tra utenti per il numero di retweet tra essi? Ha senso?

In [84]:
rt_tweets=tweets_2[tweets_2.retweet==True][['user_id','user_id_retweet','author','colours']]

In [85]:

tweets_ids=rt_tweets[['user_id','author','colours']]
retwitted_ids=retweetted[['author_id','author','colours']].rename(columns={'author_id':'user_id'})
rt_tweets_ids=pd.concat([tweets_ids,retwitted_ids]).drop_duplicates(['user_id','author']).reset_index(drop=True)

In [86]:
Nodes_df=pd.DataFrame({'Id':rt_tweets_ids.user_id,'Label':rt_tweets_ids.author,'Interval':None,'Colour':rt_tweets_ids.colours})

In [87]:
Edges_df=pd.DataFrame({'Source':rt_tweets.user_id,'Target':rt_tweets.user_id_retweet,'Type':'Directed','Kind':None,'Id':range(1, len(rt_tweets)+1),'Label':None,'Interval':None,'Weight':1.0})

In [88]:
Nodes_df.to_csv('Nodes_df5.csv', index=False)
Edges_df.to_csv('Edges_df5.csv', index=False)

##### Grafi con NetworkX

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
Edges_df=pd.read_csv("C:/Users/dvd_m/Desktop/Social Media Analytics/SMA-Project/Edges_df.csv")
G = nx.from_pandas_edgelist(Edges_df, source='Source', target='Target')
df_cluster=pd.DataFrame({"nodes":nx.clustering(G).keys(),"cluster":nx.clustering(G).values()})
grafo=pd.merge(Edges_df,df_cluster, left_on="Source",right_on="nodes")